Задача тематическое моделирование

продолжаем исследование датасета с твитами

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

как альтернатива можно скачать данные из Роспотребнадзора
https://zpp.rospotrebnadzor.ru/Forum/Appeals
для этого берём ноутбук parse_rospotrebnadzor.ipynb
устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

что надо сделать
1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики


In [48]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [49]:
colnames = ['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep', 'trtw', 'tfav', 'tstcount', 'tfoll', 'tfrien', 'listcount']

In [50]:
data_df = pd.concat([pd.read_csv('positive.csv', sep=';', header=None, names=colnames, encoding='utf-8'),
                     pd.read_csv('negative.csv', sep=';', header=None,  names=colnames, encoding='utf-8')],
                     ignore_index=True
                   )

In [51]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         226834 non-null  int64 
 1   tdate      226834 non-null  int64 
 2   tmane      226834 non-null  object
 3   ttext      226834 non-null  object
 4   ttype      226834 non-null  int64 
 5   trep       226834 non-null  int64 
 6   trtw       226834 non-null  int64 
 7   tfav       226834 non-null  int64 
 8   tstcount   226834 non-null  int64 
 9   tfoll      226834 non-null  int64 
 10  tfrien     226834 non-null  int64 
 11  listcount  226834 non-null  int64 
dtypes: int64(10), object(2)
memory usage: 20.8+ MB


In [52]:
data_df.head()

,id,tdate,tmane,ttext,ttype,trep,trtw,tfav,tstcount,tfoll,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [53]:
import pymorphy2
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from multiprocessing import Pool



In [54]:
nltk.download('stopwords')
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chinyaev.av\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:

stopwords_list = stopwords.words('russian')

In [56]:
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]

def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

#def remove_pattern(data: pd.DataFrame):
#    return list(map(lambda x: re.sub(r"@[\w]*", " ", x), data['ttext'].values))


In [58]:
data_df['ttext'] = list(map(lambda x: re.sub(r"@[\w]*", " ", x), data_df['ttext'].values))
data_df['ttext'] = list(map(lambda x: re.sub(r"http.*:.*", " ", x), data_df['ttext'].values))


In [60]:
stopwords_list.append('rt')

In [62]:
print(preprocess(data_df.ttext.iloc[2323]))

['задрот', 'perfectworld', 'день', 'пройти', 'игра', 'готовиться', 'ясказаланетэкзамен']


In [63]:
pre_data = list(tqdm(map(preprocess, data_df['ttext']), total=len(data_df)))

  0%|          | 0/226834 [00:00<?, ?it/s]

In [64]:
import csv
with open(r'pre_data_list.csv', 'w', encoding="utf-8") as fp:
#    for item in pre_data:
#        # write each item on a new line
#        fp.write("%s\n" % item)
#    print('Done')
    w = csv.writer(fp, delimiter =";")
    w.writerows(pre_data)


In [65]:
data_df['preproc_text'] = pre_data

In [66]:
data_df.head()

,id,tdate,tmane,ttext,ttype,trep,trtw,tfav,tstcount,tfoll,tfrien,listcount,preproc_text
0,408906692374446080,1386325927,pleease_shut_up,"хоть я и школота, но поверь, у нас то же сам...",1,0,0,0,7569,62,61,0,"[школотый, поверь, самый, общество, профилиров..."
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2,"[всё, таки, немного, похожий, мальчик, всё, ра..."
2,408906695083954177,1386325927,EvgeshaRe,RT : Ну ты идиотка) я испугалась за тебя!!!,1,0,1,0,1273,26,27,0,"[идиотка, испугаться]"
3,408906695356973056,1386325927,ikonnikova_21,"RT : ""Кто то в углу сидит и погибает от голод...",1,0,1,0,1549,19,17,0,"[угол, сидеть, погибать, голод, ещё, порция, в..."
4,408906761416867842,1386325943,JumpyAlex,"Вот что значит страшилка :D\nНо блин,посмотр...",1,0,0,0,597,16,23,1,"[значит, страшилка, блин, посмотреть, всё, час..."


In [67]:
from gensim.models import *
from gensim import corpora

In [68]:
dictionary = corpora.Dictionary(data_df['preproc_text'])

dictionary.filter_extremes(no_below = 10, no_above = 0.9) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 
dictionary.save('lenta.dict')

In [69]:
data_df.ttext.iloc[226829]

'Но не каждый хочет что то исправлять:(  '

In [71]:
corpus = [dictionary.doc2bow(text) for text in data_df['preproc_text']]
corpora.MmCorpus.serialize('lenta.model', corpus)

In [72]:
%%time 
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=20, chunksize=50, update_every=1, passes=2)

Wall time: 5min 39s


In [73]:
lda.show_topics(num_topics=20, num_words=10, formatted=True)

[(0,
  '0.000*"индивидуально" + 0.000*"дополнительный" + 0.000*"выборы" + 0.000*"представитель" + 0.000*"отличник" + 0.000*"проести" + 0.000*"седьмой" + 0.000*"минусовый" + 0.000*"обнимка" + 0.000*"джина"'),
 (1,
  '0.433*"ночь" + 0.212*"гулять" + 0.093*"свет" + 0.057*"праздник" + 0.052*"вечером" + 0.041*"огонь" + 0.016*"глянуть" + 0.000*"час" + 0.000*"пол" + 0.000*"пойти"'),
 (2,
  '0.409*"блин" + 0.279*"писать" + 0.210*"хотеться" + 0.038*"хуйня" + 0.027*"всякий" + 0.000*"спать" + 0.000*"делать" + 0.000*"вообще" + 0.000*"час" + 0.000*"написать"'),
 (3,
  '0.685*"очень" + 0.126*"работать" + 0.115*"спасибо" + 0.026*"главное" + 0.000*"холодно" + 0.000*"скучать" + 0.000*"жаль" + 0.000*"плохо" + 0.000*"улица" + 0.000*"жалко"'),
 (4,
  '0.258*"который" + 0.247*"время" + 0.157*"работа" + 0.120*"хотя" + 0.065*"история" + 0.051*"русский" + 0.026*"зря" + 0.023*"рассказать" + 0.014*"шутить" + 0.000*"делать"'),
 (5,
  '0.000*"индивидуально" + 0.000*"дополнительный" + 0.000*"выборы" + 0.000*"предс

In [74]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [78]:
%%time 
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

Wall time: 12.5 s


In [40]:
vis_data.to_json

<bound method PreparedData.to_json of PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.194663 -0.220135       1        1  18.410596
5     -0.287954 -0.017436       2        1  17.078785
3     -0.130777 -0.181214       3        1  16.436754
6      0.128129  0.268808       4        1  13.530467
4      0.193231 -0.165205       5        1  12.149869
2     -0.212225  0.075229       6        1  11.893536
1      0.114933  0.239954       7        1  10.499993, topic_info=         Term          Freq         Total Category  logprob  loglift
39        это  23795.000000  23795.000000  Default  30.0000  30.0000
24     хотеть  18840.000000  18840.000000  Default  29.0000  29.0000
15       http  24887.000000  24887.000000  Default  28.0000  28.0000
100      мочь  11475.000000  11475.000000  Default  27.0000  27.0000
18        ещё  11639.000000  11639.000000  Default  26.0000  26.0000
...       ...      

In [ ]:
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

In [ ]:
coherence_model_lda = CoherenceModel(model=lda, texts=data_df['ttext'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Средняя когерентность: ', coherence_lda)

In [ ]:
lda.sh